In [2]:
from cvzone.HandTrackingModule import HandDetector
import cv2
import os
import numpy as np

print("Warning: The presentation should be in 1280x720 resolution.")
print("Warning: Presentation file names should be marked or named as numbers in the order they need to be presented.")
print("Warning: The image file extensions must be .png, .jpg, or .jpeg only.")

# Parameters
width, height = 1280, 720
gestureThreshold = 300

# User Input for Presentation Path
folderPath = input("Enter the path to the presentation folder: ")

# Verify the folder path is valid
if not os.path.isdir(folderPath):
    print("Invalid folder path!")
    exit()

# Get list of presentation images
validExtensions = (".jpg", ".png", ".jpeg")
pathImages = [file for file in os.listdir(folderPath) if file.lower().endswith(validExtensions)]
pathImages = sorted(pathImages, key=lambda x: int(os.path.splitext(x)[0]))
if not pathImages:
    print("No valid image files found in the folder!")
    exit()

# Camera Setup
cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

# Hand Detector
detectorHand = HandDetector(detectionCon=0.8, maxHands=1)

# Variables
delay = 30
buttonPressed = False
counter = 0
imgNumber = 0
annotations = [[]]
annotationNumber = -1
annotationStart = False
hs, ws = int(120 * 1), int(213 * 1)  # width and height of small image

while True:
    # Get image frame
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break
    img = cv2.flip(img, 1)
    pathFullImage = os.path.join(folderPath, pathImages[imgNumber])
    imgCurrent = cv2.imread(pathFullImage)
    if imgCurrent is None:
        print(f"Failed to load image: {pathFullImage}")
        break

    # Find the hand and its landmarks
    hands, img = detectorHand.findHands(img)  # with draw
    # Draw Gesture Threshold line
    cv2.line(img, (0, gestureThreshold), (width, gestureThreshold), (0, 255, 0), 10)

    if hands and not buttonPressed:  # If hand is detected
        hand = hands[0]
        cx, cy = hand["center"]
        lmList = hand["lmList"]  # List of 21 Landmark points
        fingers = detectorHand.fingersUp(hand)  # List of which fingers are up

        # Constrain values for easier drawing
        xVal = int(np.interp(lmList[8][0], [width // 2, width], [0, width]))
        yVal = int(np.interp(lmList[8][1], [150, height - 150], [0, height]))
        indexFinger = (xVal, yVal)

        if cy <= gestureThreshold:  # If hand is at the height of the face
            if fingers == [1, 0, 0, 0, 0]:  # Gesture 1 = Back
                print("Left")
                buttonPressed = True
                if imgNumber > 0:
                    imgNumber -= 1
                    annotations = [[]]
                    annotationNumber = -1
                    annotationStart = False
            elif fingers == [0, 0, 0, 0, 1]:  # Gesture 2 = Front
                print("Right")
                buttonPressed = True
                if imgNumber < len(pathImages) - 1:
                    imgNumber += 1
                    annotations = [[]]
                    annotationNumber = -1
                    annotationStart = False

        if fingers == [0, 1, 1, 0, 0]:  # Gesture 3 = Pointer
            cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)
        elif fingers == [0, 1, 0, 0, 0]:  # Gesture 4 = Draw
            if not annotationStart:
                annotationStart = True
                annotationNumber += 1
                annotations.append([])
            annotations[annotationNumber].append(indexFinger)
            cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)
        elif fingers == [0, 1, 1, 1, 0]:  # Gesture 5 = Erase
            if annotations:
                annotations.pop(-1)
                annotationNumber -= 1
                buttonPressed = True
        else:
            annotationStart = False

    if buttonPressed:
        counter += 1
        if counter > delay:
            counter = 0
            buttonPressed = False

    for i, annotation in enumerate(annotations):
        for j in range(len(annotation)):
            if j != 0:
                cv2.line(imgCurrent, annotation[j - 1], annotation[j], (0, 0, 200), 12)

    imgSmall = cv2.resize(img, (ws, hs))
    imgCurrent[0:hs, width - ws: width] = imgSmall

    cv2.imshow("Slides", imgCurrent)
    cv2.imshow("Image", img)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Enter the path to the presentation folder: Presentation
Left
Left
Left
Left
Left
Left


KeyboardInterrupt: 